# Paths and download data

This define the path where data are stored on your local machine (where you have downloaded them) and where the processed data will be stored while running the others notebooks (i.e in the same folder).

# Paths 


### Original (raw) data in the un-zip file: `PlaineMorteLake_hydro_thermodynamics`

In [39]:
#data
path_data = joinpath(dirname(dirname(pwd())),"data")*"/" #dirname go back in your folder tree

#sub folder in data/
# CTD
path_CTD =  joinpath(path_data, "CTD") *"/"
#DC22
path_DC22 = joinpath(path_data, "DC22") *"/"
#lake data
path_lake = joinpath(path_data, "lake_data") *"/";

### Local directory of processed data (products) on your own machine

In [41]:
#products main folder
path_products =  joinpath(dirname(dirname(pwd())),"products/")
#sub-folder in products/
path_plots = path_products *"plots/";
path_results_cali_cond = path_plots * "calibration_conductivity/";
path_results_cali_temp = path_plots * "calibration_temperature/";

#Create products folder tree, if not already existing
mkpath(path_products)
mkpath(path_plots)
mkpath(path_results_cali_cond)
mkpath(path_results_cali_temp);
